# **WELCOME TO THIS NOTEBOOK**

Let's install pyspark

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 53.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=be11966c287b9d77b5b0cc36b670399fd2570bdd3cb21f926c64d9acc8554468
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Importing the modules

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [4]:
spark = SparkSession.builder.appName('lastfm').getOrCreate()

# Loading the dataset

In [5]:
file_path = '/content/drive/MyDrive/dataset/listenings.csv'
df_listenings = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path)
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Cleaning tables 

In [6]:
df_listening = df_listenings.drop('date')
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

In [7]:
df_listenings = df_listenings.na.drop()
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

In [8]:
row_numbers = df_listenings.count()
column_numbers = len(df_listenings.columns)
print(row_numbers,column_numbers)

13758905 5



# Let's Perform some aggregation
to see how many times each user has listened to specific track


In [9]:
df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id','track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|Chelsea Hotel - L...|    1|
| --Seph|          Paris 2004|    7|
| --Seph|Airplanes [feat H...|    1|
| --Seph|              Monday|    1|
| --Seph|         The Embrace|    1|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|       Life On Mars?|    1|
| --Seph|Belina (Original ...|    1|
| --Seph| White Winter Hymnal|    3|
| --Seph|               Leloo|    1|
| --Seph|      Hour for magic|    2|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph|     The Way We Were|    1|
| --Seph| Air on the G String|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph|        Window Blues|    1|
| --Seph|  California Waiting|    1|
| --Seph|       Phantom Pt II|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph|   Summa for Strings|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [10]:
n_agg_rows = df_listenings_agg.count()
n_agg_columns = len(df_listenings_agg.columns) 
print(n_agg_rows,n_agg_columns)

9930128 3


# Let's convert the user id and track columns into unique integers




In [11]:
df_listenings_agg = df_listenings_agg.limit(20000)

In [21]:
, StringIndexerModel
indexer = [StringIndexer(inputCol = col, outputCol=col+'_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)-set(['count']))]
pipeline = Pipeline(stages=indexer)
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()
data = data.select('user_id_index', 'track_index', 'count').orderBy('user_id_index')
data.show()

+-------+--------------------+-----+-------------+-----------+
|user_id|               track|count|user_id_index|track_index|
+-------+--------------------+-----+-------------+-----------+
| --Seph|          Nightmares|    1|         69.0|    10600.0|
| --Seph|Virus (Luke Fair ...|    1|         69.0|    15893.0|
| --Seph|Airplanes [feat H...|    1|         69.0|      521.0|
| --Seph|Belina (Original ...|    1|         69.0|     3280.0|
| --Seph|              Monday|    1|         69.0|      334.0|
| --Seph|Hungarian Dance No 5|    1|         69.0|     7555.0|
| --Seph|       Life On Mars?|    1|         69.0|     1164.0|
| --Seph|  California Waiting|    1|         69.0|      195.0|
| --Seph|       Phantom Pt II|    1|         69.0|     1378.0|
| --Seph|   Summa for Strings|    1|         69.0|    13737.0|
| --Seph|      Hour for magic|    2|         69.0|     7492.0|
| --Seph|Hungarian Rhapsod...|    1|         69.0|     7556.0|
| --Seph|     The Way We Were|    1|         69.0|    1

# Train and Test data

In [14]:
(training,test)=data.randomSplit([0.5,0.5])

# Let's Create our Model

In [15]:
USERID='user_id_index'
TRACK = 'track_index'
COUNT = 'count'
als = ALS(maxIter=5,regParam=0.01,userCol=USERID,itemCol=TRACK,ratingCol=COUNT)
model = als.fit(training)

prediction = model.transform() 

Py4JJavaError: ignored


# Generate top 10 Track recommendations for each user

In [ ]:
recs = model.recommendForAllUsers(10)

In [ ]:
recs.take(1)